In [26]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import plotly
import plotly.graph_objs as go
import plotly.express as px
from random import randint
import colorsys 
import os
import ipywidgets as widgets

In [27]:
def getneighbor_grades (reference_grade, countries, quality_criteria, k=5):
    ''' This is the main function, prepare the data and headings, set the
    model and perform the predictions
    
    Arguments
        reference_grade: reference grade name;
        countries: is a list with the selected countries to be used as reference for
            the grades comparison.
        quality_criteria: list with the selected crude quality criteria;
        k: number of nearest grades to be returned, including the reference grade as 1st item

    Return:
        1. a list of lists with the data of the k nearest grades found
        2. a list of the grades eligible for comparison
    '''
    # read the data file
    crude_data_df = pd.read_csv(datapath)
    
    # Get the parameters that will be used for the comparison
    limit =  len(quality_criteria)
    listParameters = []
    col_names = ['Crude']
    for i in range(0, limit):
        if quality_criteria[i] == 'SULPHUR':
            listParameters.append('Sulphur (%)')
            col_names.append('Sulphur (%)')

        elif quality_criteria[i] == 'TAN':
            listParameters.append('Tan (mgKOH/g)')
            col_names.append('Tan (mgKOH/g)')            
        
        else:
            listParameters.append(quality_criteria[i])
            col_names.append(quality_criteria[i])

    # include additional columns
    col_names.append('Country')
    col_names.append('Notes')
    
    df1 = crude_data_df[col_names]


    if 'Tan (mgKOH/g)' in df1.columns:
        #Remove the rows containing NaN in the TAN column
        df2 = df1[df1['Tan (mgKOH/g)'].notna()]
        df2.reindex()
    else:
        df2 = df1

    results = []

    #Prepare the items for the reference grade
    ref_grade = df2.loc[df2['Crude'] == reference_grade]
    ref_param = ref_grade[listParameters].values.tolist()
    
    #Filter the sample grades using the selected reference countries
    df2 = df2.loc[df2['Country'].isin(countries)]

    #In case the country of the reference grade is not chosen for the selection
    #criteria, all the data of the reference grade need to be added back into the
    #filtered dataframe (otherwise it will note be shown in the results)!
    if ref_grade['Country'].iloc[0] not in countries:
        df2 = df2.append(ref_grade)

    # Take the values of all the existing grades for training, excluding the label, 
    # which is a string
    samples = df2[listParameters].values.tolist()

    # Need to verify that the sample size is bigger than k
    if len(samples) < k:
        k = len(samples)

    # Apply the K NearestNeighbors model
    neigh = NearestNeighbors(n_neighbors=k)
    # Train the model
    neigh.fit(samples)
    # Find the nearest elements to the reference grade
    distance, indices = neigh.kneighbors(ref_param)
    nGrade_values = []

    # add the grades on the results list
    for i in indices[0]:
        nGrade_values = df2.iloc[i].values.tolist()
        results.append(nGrade_values)
 
    #If the reference_grade is not the 1st in the list, then move its position
    #This may be required when reindex is used above
    if results[0][0] != reference_grade:
        for i in range(len(results)):
            if results[i][0] == reference_grade:
                temp = results[i]
                results.remove(temp)
                results.insert(0, temp)

    #Insert the ranking value for the 'Ranking' column
    for i in range(len(results)):
        results[i].insert(0, i)
        
    # update the table header
    col_names.insert(0, 'Ranking')
    
    return results, col_names

In [28]:
def figures_data (results, headings):
    """ Prepare the results to be used for the chart(s)
    Args
        results: A list with the result values
        headings: A list with the result headings for the table
    Returns
        resList: A list of dictionaries with the corresponding values for each grade
    """
    
    resList = []

    for grade in results:
        temp_dic = { }        
        for item in headings:
            if 'Crude' in item:
                temp_dic['Crude'] = grade[headings.index('Crude')]
            if 'API' in item:
                temp_dic['API'] = grade[headings.index('API')]
            if 'Sulphur (%)' in item:
                temp_dic['Sulphur (%)'] = grade[headings.index('Sulphur (%)')]
            if 'TAN' in item:
                temp_dic['TAN'] = grade[headings.index('TAN')]

        resList.append(temp_dic)

    return resList

In [29]:
def get_figures_px(df, features, nrGrades):
    """ Creates visualizations using plotly.express 

    Args
        df: A pandas dataframe with the data to plot;
        features: List of features to extract;
        nrGrades: Nr of points to display;

    Returns
        fig: list containing plotly visualizations
    """
    color = ['rgb(255,0,0)', 'rgb(255,200,200)', 'rgb(255,100,0)', 'rgb(255,150,0)', 
             'rgb(255,200,0)', 'rgb(255,255,0)', 'rgb(200,255,0)', 'rgb(100,255,0)', 
             'rgb(0,255,255)', 'rgb(0,200,255)', 'rgb(0,100,255)', 'rgb(0,0,255)', 
             'rgb(100,0,255)', 'rgb(200,0,255)', 'rgb(188, 176, 182)', 'rgb(142, 137, 140)', 
             'rgb(205, 24, 118)', 'rgb(24, 177, 205)', 'rgb(0, 0, 0)', 'rgb(170, 144, 60)']

    nrFeatures = len(features)    
    
    if nrFeatures == 1:
        crude_col = df['Crude'].tolist()
        feature_col = df[features[0]].tolist()
        fig=px.bar(df,x=crude_col,y=feature_col,
                color = 'Crude',
                color_discrete_sequence=color,
                labels={'y':features[0], 'x':'[Grades]'},
                )

    if nrFeatures == 2:
        fig = px.scatter(
            data_frame=df, 
            x=features[1], 
            y=features[0],
            color="Crude", 
            color_discrete_sequence=color,
            # text="Crude"
        )
        fig.update_traces(
            marker=dict(size=12, line=dict(width=2,)), 
            selector=dict(mode='markers')
        )       

    if nrFeatures == 3:
        fig = px.scatter_3d(
            data_frame=df,
            x='Sulphur (%)',
            y='Tan (mgKOH/g)',
            z='API',
            color='Crude',
            color_discrete_sequence=color
        )

    return fig


In [30]:
def get_figs_clusters(df):
    color = ['rgb(255,0,0)', 'rgb(255,200,200)', 'rgb(255,100,0)', 'rgb(255,150,0)', 
             'rgb(255,200,0)', 'rgb(255,255,0)', 'rgb(200,255,0)', 'rgb(100,255,0)', 
             'rgb(0,255,255)', 'rgb(0,200,255)', 'rgb(0,100,255)', 'rgb(0,0,255)', 
             'rgb(100,0,255)', 'rgb(200,0,255)', 'rgb(188, 176, 182)', 'rgb(142, 137, 140)', 
             'rgb(205, 24, 118)', 'rgb(24, 177, 205)', 'rgb(0, 0, 0)', 'rgb(170, 144, 60)']
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['Sulphur (%)'],
        y=df['API'],
        mode="markers+text",
        marker=dict(
            color=colors,
        )
    ))
    fig.add_vrect(
        x0=-0.01, 
        x1=0.5, 
        line_width=0, 
        fillcolor="red", 
        opacity=0.2,
    )
    fig.add_hline(y=10, line_width=3, line_dash="dash", line_color="black")
    fig.add_hline(y=22.3, line_width=3, line_dash="dash", line_color="green")
    fig.add_hline(y=32.1, line_width=3, line_dash="dash", line_color="red")
    fig.add_hline(y=42.1, line_width=3, line_dash="dash", line_color="yellow")
    fig.add_annotation(x=5, y=15, text="HEAVY", showarrow=False, yshift=10)
    fig.add_annotation(x=5, y=25, text="MEDIUM", showarrow=False, yshift=10)
    fig.add_annotation(x=5, y=35, text="LIGHT", showarrow=False, yshift=10)
    fig.add_annotation(x=5, y=45, text="EXTRA-LIGHT", showarrow=False, yshift=10)
    fig.update_layout(
        title="All Grades in Dataset",
        xaxis_title="Sulphur (%)",
        yaxis_title="°API",
    )

    return fig

In [31]:
def get_setting_list(parameter):
    '''
    Return a list with available Grades or Countries, depending
    on the selected parameter
    '''

    df1 = pd.read_csv(datapath)
    #Get the list of available/eligible grades
    df_list = df1[parameter].sort_values()
    df_list = df_list.drop_duplicates()
    df_list = df_list.dropna()
    
    return df_list.values.tolist()

In [32]:
# Read data
workingpath = os.getcwd()
datapath = workingpath + '/data/master_crude_20220811.csv'
df_data = pd.read_csv(datapath)

In [33]:
countries = get_setting_list('Country')
all_grades = get_setting_list('Crude')

In [34]:
# ====== Set up Input parameters ============

# Initialise the dropdown for Reference grade
dropdown_refgrade = widgets.Dropdown(
    options=all_grades,
    description='Ref. Grade:',
    value='BRENT'
)

# Initialise the dropdown list for Countries
list_countries = widgets.SelectMultiple(
    options=countries,
    description='Countries:',
    disabled=False
)

# Initialise the dropdown list for Quality properties
quality_properties = widgets.SelectMultiple(
    options=['API', 'Sulphur (%)', 'Tan (mgKOH/g)'],
    value=['API', 'Sulphur (%)'],
    description='Properties:',
    disabled=False
)

# Initialise slider to select number of grades
kvalues = widgets.IntSlider(
    min=1,
    max=20,
    step=1,
    description='Nr. Grades:',
    value=3
)

In [35]:
# ====== Set up Output ============

output = widgets.Output()

def output_results(reference_grade, countries, quality_criteria, kvalue):
    output.clear_output()
    results, header = getneighbor_grades(reference_grade, countries, quality_criteria, kvalue)
    df_result = pd.DataFrame(results, columns=header)
    with output:
        display(df_result)

def btn_eventhandler(obj):
    output.clear_output()
    results, header = getneighbor_grades(
                dropdown_refgrade.value, 
                list_countries.value, 
                quality_properties.value, 
                kvalues.value)
    df_result = pd.DataFrame(results, columns=header)
    chart = get_figures_px(df_result, quality_properties.value, kvalues.value)
    display(df_result.style.hide_index())
    display(chart)
    
def btn_cluster_eventhandler(obj):
    chart = get_figs_clusters(df_data)
    display(chart)
#===================================================
btn_submit = widgets.Button(description='Submit')
btn_submit.on_click(btn_eventhandler)
#
btn_cluster = widgets.Button(description='Show Clusters')
btn_cluster.on_click(btn_cluster_eventhandler)

In [37]:
# === Interface Dashboard: Input & Output =============
input_widgets_row1 = widgets.HBox(
[dropdown_refgrade, list_countries, quality_properties])

input_widgets_row2 = widgets.HBox(
[kvalues, btn_submit, btn_cluster])

display(input_widgets_row1)
display(input_widgets_row2)